In [31]:
# load json file as dictionary
import json
import sys
import pandas as pd
import numpy as np
import re

from umap import UMAP
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
import bertopic
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech
from bertopic.vectorizers import ClassTfidfTransformer
import datamapplot
import nbformat


In [3]:
unseen_data_labelled = pd.read_csv("unseen_data_predictions_20240704-114801.csv")
patents_text = pd.read_csv("patents_text_deduplicated.csv")
training_data = pd.read_csv("training_data_new3.csv")

/var/folders/jy/j77v476x03737ryp7gll_0p00000gn/T/ipykernel_1094/1228290111.py:2: DtypeWarning: Columns (18,20,21,23,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  patents_text = pd.read_csv("patents_text_deduplicated.csv")


In [4]:
len(patents_text)

20445

In [5]:
len(unseen_data_labelled.publication_number_EPO)

19457

In [6]:
# Create dictionaries for quick lookup, handling non-unique indices
unseen_data_dict = unseen_data_labelled.drop_duplicates(subset='publication_number_EPO').set_index('publication_number_EPO')[['predicted_SofAI', 'predicted_AIforS']].to_dict(orient='index')
training_data_dict = training_data.drop_duplicates(subset='publication_number_EPO').set_index('publication_number_EPO')[['SofAI', 'AIforS']].to_dict(orient='index')

# Function to determine SofAI and AIforS values
def get_values(pub_number):
    if pub_number in unseen_data_dict:
        return unseen_data_dict[pub_number]['predicted_SofAI'], unseen_data_dict[pub_number]['predicted_AIforS']
    elif pub_number in training_data_dict:
        return training_data_dict[pub_number]['SofAI'], training_data_dict[pub_number]['AIforS']
    else:
        return None, None

# Apply the function to the patents_text dataframe
patents_text['SofAI'], patents_text['AIforS'] = zip(*patents_text['publication_number_EPO'].apply(get_values))

In [7]:
# Count the occurrences for each category
only_SofAI = ((patents_text['SofAI'] == 1) & (patents_text['AIforS'] == 0)).sum()
only_AIforS = ((patents_text['SofAI'] == 0) & (patents_text['AIforS'] == 1)).sum()
both = ((patents_text['SofAI'] == 1) & (patents_text['AIforS'] == 1)).sum()
none = ((patents_text['SofAI'] == 0) & (patents_text['AIforS'] == 0)).sum()

# Print the counts
print(f"Number of rows with only SofAI set to 1: {only_SofAI}")
print(f"Number of rows with only AIforS set to 1: {only_AIforS}")
print(f"Number of rows with both SofAI and AIforS set to 1: {both}")
print(f"Number of rows with neither SofAI nor AIforS set to 1: {none}")


Number of rows with only SofAI set to 1: 548
Number of rows with only AIforS set to 1: 17302
Number of rows with both SofAI and AIforS set to 1: 2500
Number of rows with neither SofAI nor AIforS set to 1: 95


In [9]:
patents_text.to_csv("patents_text_topic_modelling2.csv", index=False)